In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List

load_dotenv()
RESUME_PATH = "data/resume.pdf" 

class ResumeProfile(BaseModel):
    full_name: str = Field(description="The full name of the candidate.")
    
    target_role: str = Field(description="The specific job title the candidate is aiming for based on their summary/bio (e.g. 'Data Scientist', 'Frontend Dev').")
    
    location: str = Field(description="Candidate's city/country. Return 'Remote' if not specified.")
    summary: str = Field(description="A brief summary of the candidate's profile.")
    technical_skills: List[str] = Field(description="List of specific technical tools, languages, and frameworks.")
    experience_years: int = Field(description="Estimated total years of professional experience.")

def analyze_resume(file_path: str) -> ResumeProfile:
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    resume_text = "\n".join([doc.page_content for doc in docs])
    
    print("🧠 Analyzing resume for Target Role, Name & Skills...")
    llm = ChatOpenAI(model="gpt-4o", temperature=0)
    structured_llm = llm.with_structured_output(ResumeProfile)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a Career Advisor. Analyze the resume to find the specific role the candidate wants."),
        ("human", "{text}")
    ])
    return (prompt | structured_llm).invoke({"text": resume_text})

if os.path.exists(RESUME_PATH):
    profile = analyze_resume(RESUME_PATH)
    print(f"\n✅ Profile Loaded!")
    print(f"👤 Name:        {profile.full_name}")         
    print(f"🎯 Target Role: {profile.target_role}") 
    print(f"📍 Location:    {profile.location}")
    print(f"🛠️ Skills:      {', '.join(profile.technical_skills)}") 

🧠 Analyzing resume for Target Role, Name & Skills...

✅ Profile Loaded!
👤 Name:        John Huber
🎯 Target Role: UX Designer
📍 Location:    New York, USA
🛠️ Skills:      HTML5, CSS, JavaScript, Adobe Suite, Sketch, InVision, Balsamiq
